In [1]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"

In [2]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

import numpy as np

import cupy
import cudf

/conda/envs/rapids/lib/python3.7/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/conda/envs/rapids/lib/python3.7/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice/.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))


In [3]:
BASE_DIR = '/raid/'
PREFIX_in = 'nv' # org, dask, nv, pd
PREFIX_out = 'nv3' # nv

In [4]:
cluster = LocalCUDACluster(
    protocol="tcp",
    rmm_pool_size="26GB"
)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:46701 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 429.50 GB


In [5]:
train = dask_cudf.read_parquet(BASE_DIR + PREFIX_in + '_' + PREFIX_out + '_out_train/*.parquet')
valid = dask_cudf.read_parquet(BASE_DIR + PREFIX_in + '_' + PREFIX_out + '_out_valid/*.parquet')

In [6]:
train.columns

Index(['timestamp', 'a_follower_count', 'a_following_count',
       'b_follower_count', 'b_following_count', 'hashtags_count_t',
       'domains_count_t', 'links_count_t', 'hashtags', 'tweet_id', 'media',
       'links', 'domains', 'tweet_type', 'language', 'a_user_id',
       'a_is_verified', 'b_user_id', 'b_is_verified', 'b_follows_a', 'dt_dow',
       'reply', 'retweet', 'retweet_comment', 'like', 'TE_media_reply',
       'TE_media_retweet', 'TE_media_retweet_comment', 'TE_media_like',
       'TE_tweet_type_reply', 'TE_tweet_type_retweet',
       'TE_tweet_type_retweet_comment', 'TE_tweet_type_like',
       'TE_language_reply', 'TE_language_retweet',
       'TE_language_retweet_comment', 'TE_language_like', 'TE_a_user_id_reply',
       'TE_a_user_id_retweet', 'TE_a_user_id_retweet_comment',
       'TE_a_user_id_like', 'TE_b_user_id_reply', 'TE_b_user_id_retweet',
       'TE_b_user_id_retweet_comment', 'TE_b_user_id_like',
       'TE_domains_language_b_follows_a_tweet_type_media_a_is

In [7]:
features = [
    'media', 'tweet_type', 'language',
    'a_follower_count', 'a_following_count', 'a_is_verified', 
    'b_follower_count', 'b_following_count', 'b_is_verified', 'b_follows_a',
    'hashtags_count_t', 'domains_count_t', 'links_count_t',
    'TE_media_reply', 'TE_media_retweet', 'TE_media_retweet_comment', 'TE_media_like',
    'TE_tweet_type_reply', 'TE_tweet_type_retweet',
    'TE_tweet_type_retweet_comment', 'TE_tweet_type_like',
    'TE_language_reply', 'TE_language_retweet',
    'TE_language_retweet_comment', 'TE_language_like', 'TE_a_user_id_reply',
    'TE_a_user_id_retweet', 'TE_a_user_id_retweet_comment',
    'TE_a_user_id_like', 'TE_b_user_id_reply', 'TE_b_user_id_retweet',
    'TE_b_user_id_retweet_comment', 'TE_b_user_id_like',
    'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_reply',
    'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_retweet',
    'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_retweet_comment',
    'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_like',
    'media_count', 'tweet_type_count', 'language_count', 'a_user_id_count',
    'b_follower_count_DifferenceLag', 'b_following_count_DifferenceLag', 'language_DifferenceLag'
    'dt_dow', 'timestamp_to_datetime_to_hour', 
    'timestamp_to_datetime_to_minute', 'timestamp_to_datetime_to_second'
]
features = [x for x in features if x in train.columns]
label_names = ['reply', 'retweet', 'retweet_comment', 'like']
other = ['tweet_id']
print(features, len(features))

['media', 'tweet_type', 'language', 'a_follower_count', 'a_following_count', 'a_is_verified', 'b_follower_count', 'b_following_count', 'b_is_verified', 'b_follows_a', 'hashtags_count_t', 'domains_count_t', 'links_count_t', 'TE_media_reply', 'TE_media_retweet', 'TE_media_retweet_comment', 'TE_media_like', 'TE_tweet_type_reply', 'TE_tweet_type_retweet', 'TE_tweet_type_retweet_comment', 'TE_tweet_type_like', 'TE_language_reply', 'TE_language_retweet', 'TE_language_retweet_comment', 'TE_language_like', 'TE_a_user_id_reply', 'TE_a_user_id_retweet', 'TE_a_user_id_retweet_comment', 'TE_a_user_id_like', 'TE_b_user_id_reply', 'TE_b_user_id_retweet', 'TE_b_user_id_retweet_comment', 'TE_b_user_id_like', 'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_reply', 'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_retweet', 'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_retweet_comment', 'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_like',

In [8]:
#features = [
#    'media', 'tweet_type', 'language', 
#    'a_follower_count', 'a_following_count', 'a_is_verified', 
#    'b_follower_count', 'b_following_count', 'b_is_verified', 'b_follows_a', 
#    'len_hashtags', 'len_domains', 'len_links', 
#    'TE_media_reply', 'TE_tweet_type_reply', 'TE_language_reply', 
#    'TE_a_user_id_reply', 'TE_b_user_id_reply', 
#    'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_reply', 
#    'media_count', 'tweet_type_count', 'language_count', 'a_user_id_count', 
#    'b_user_id_count', 'a_following_count_a_ff_rate', 
#    'b_following_count_b_ff_rate', 'b_follower_count_DifferenceLag', 
#    'b_following_count_DifferenceLag', 'language_DifferenceLag', 
#    'timestamp_to_datetime_to_hour', 'timestamp_to_datetime_to_minute', 'timestamp_to_datetime_to_second'
#]
#
#label_names = ['reply', 'retweet', 'retweet_comment', 'like']
#other = ['tweet_id']

In [9]:
%%time
RMV = [x for x in train.columns if not(x in features+label_names+other)]

train = train.drop(RMV,axis=1)
RMV = [c for c in RMV if c in valid.columns]
valid = valid.drop(RMV,axis=1)    
wait(train)
wait(valid)

CPU times: user 188 ms, sys: 22.4 ms, total: 210 ms
Wall time: 200 ms


DoneAndNotDoneFutures(done=set(), not_done=set())

In [10]:
train

,a_follower_count,a_following_count,b_follower_count,b_following_count,hashtags_count_t,domains_count_t,links_count_t,tweet_id,media,tweet_type,language,a_is_verified,b_is_verified,b_follows_a,reply,retweet,retweet_comment,like,TE_media_reply,TE_media_retweet,TE_media_retweet_comment,TE_media_like,TE_tweet_type_reply,TE_tweet_type_retweet,TE_tweet_type_retweet_comment,TE_tweet_type_like,TE_language_reply,TE_language_retweet,TE_language_retweet_comment,TE_language_like,TE_a_user_id_reply,TE_a_user_id_retweet,TE_a_user_id_retweet_comment,TE_a_user_id_like,TE_b_user_id_reply,TE_b_user_id_retweet,TE_b_user_id_retweet_comment,TE_b_user_id_like,TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_reply,TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_retweet,TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_retweet_comment,TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_like,media_count,tweet_type_count,language_count,a_user_id_count,timestamp_to_datetime_to_hour,timestamp_to_datetime_to_minute,timestamp_to_datetime_to_second,b_follower_count_DifferenceLag,b_following_count_DifferenceLag
npartitions=88,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,float32,float32,int32,int32,int32,int64,int64,int64,float32,bool,bool,bool,int8,int8,int8,int8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int16,int16,int16,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [11]:
SAMPLE_RATIO = 0.1
SEED = 1

if SAMPLE_RATIO < 1.0:
    train['sample'] = train['tweet_id'].map_partitions(lambda cudf_df: cudf_df.hash_encode(stop=10))
    print(len(train))
    
    train = train[train['sample']<10*SAMPLE_RATIO]
    train, = dask.persist(train)
    train.head()
    print(len(train))


Y_train = train[label_names]
Y_train, = dask.persist(Y_train)
Y_train.head()    
    
train = train.drop(['sample','tweet_id']+label_names,axis=1)
train, = dask.persist(train)
train.head()

print('Using %i features:'%(len(features)),train.shape[1])
np.asarray(features)

87914868
8743809
Using 46 features: 46


array(['media', 'tweet_type', 'language', 'a_follower_count',
       'a_following_count', 'a_is_verified', 'b_follower_count',
       'b_following_count', 'b_is_verified', 'b_follows_a',
       'hashtags_count_t', 'domains_count_t', 'links_count_t',
       'TE_media_reply', 'TE_media_retweet', 'TE_media_retweet_comment',
       'TE_media_like', 'TE_tweet_type_reply', 'TE_tweet_type_retweet',
       'TE_tweet_type_retweet_comment', 'TE_tweet_type_like',
       'TE_language_reply', 'TE_language_retweet',
       'TE_language_retweet_comment', 'TE_language_like',
       'TE_a_user_id_reply', 'TE_a_user_id_retweet',
       'TE_a_user_id_retweet_comment', 'TE_a_user_id_like',
       'TE_b_user_id_reply', 'TE_b_user_id_retweet',
       'TE_b_user_id_retweet_comment', 'TE_b_user_id_like',
       'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_reply',
       'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_retweet',
       'TE_domains_language_b_follows_a_tweet_ty

In [12]:
SAMPLE_RATIO = 0.35 # VAL SET NOW SIZE OF TEST SET
SEED = 1
if SAMPLE_RATIO < 1.0:
    print(len(valid))
    valid['sample'] = valid['tweet_id'].map_partitions(lambda cudf_df: cudf_df.hash_encode(stop=10))
    
    valid = valid[valid['sample']<10*SAMPLE_RATIO]
    valid, = dask.persist(valid)
    valid.head()
    print(len(valid))
    
Y_valid = valid[label_names]
Y_valid, = dask.persist(Y_valid)
Y_valid.head()

valid = valid.drop(['sample','tweet_id']+label_names,axis=1)
valid, = dask.persist(valid)
valid.head()

33471563
13400406


,a_follower_count,a_following_count,b_follower_count,b_following_count,hashtags_count_t,domains_count_t,links_count_t,media,tweet_type,language,...,TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_like,media_count,tweet_type_count,language_count,a_user_id_count,timestamp_to_datetime_to_hour,timestamp_to_datetime_to_minute,timestamp_to_datetime_to_second,b_follower_count_DifferenceLag,b_following_count_DifferenceLag
3,11005,10795,4.0,6.0,0,0,0,13,3,12.0,...,0.534409,54264254,51003473,13942809,10,2,45,55,0.0,0.0
4,11005,10795,4.0,6.0,2,0,0,5,3,12.0,...,0.749724,16512501,51003473,13942809,10,1,55,19,0.0,0.0
6,769176,190,2.0,29.0,0,0,0,13,3,12.0,...,0.426491,54264254,51003473,13942809,5191,4,41,19,0.0,0.0
7,1344732,125,2.0,29.0,0,0,0,1,3,55.0,...,0.414979,958944,51003473,37724583,3733,18,30,0,0.0,0.0
10,56,89,99.0,656.0,0,0,0,7,3,12.0,...,0.785251,6650374,51003473,13942809,5,4,28,7,0.0,0.0


In [13]:
import xgboost as xgb
print('XGB Version',xgb.__version__)

xgb_parms = { 
    'max_depth':8, 
    'learning_rate':0.1, 
    'subsample':0.8,
    'colsample_bytree':0.3, 
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'tree_method':'gpu_hist',
    'predictor' : 'gpu_predictor'
}

XGB Version 1.2.0


In [14]:
if train.columns.duplicated().sum()>0:
    raise Exception(f'duplicated!: { train.columns[train.columns.duplicated()] }')
print('no dup :) ')
print(f'X_train.shape {train.shape}')
print(f'X_valid.shape {valid.shape}')

no dup :) 
X_train.shape (Delayed('int-295c52e7-eb43-4bd6-8202-fd1a53701b30'), 46)
X_valid.shape (Delayed('int-e348a716-cd86-4fdf-b9da-f2b6ce046b0b'), 46)


In [15]:
for col in train.columns:
    if train[col].dtype=='bool':
        train[col] = train[col].astype('int8')
        valid[col] = valid[col].astype('int8')
train,valid = dask.persist(train,valid)
train.head()
valid.head()

,a_follower_count,a_following_count,b_follower_count,b_following_count,hashtags_count_t,domains_count_t,links_count_t,media,tweet_type,language,...,TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_like,media_count,tweet_type_count,language_count,a_user_id_count,timestamp_to_datetime_to_hour,timestamp_to_datetime_to_minute,timestamp_to_datetime_to_second,b_follower_count_DifferenceLag,b_following_count_DifferenceLag
3,11005,10795,4.0,6.0,0,0,0,13,3,12.0,...,0.534409,54264254,51003473,13942809,10,2,45,55,0.0,0.0
4,11005,10795,4.0,6.0,2,0,0,5,3,12.0,...,0.749724,16512501,51003473,13942809,10,1,55,19,0.0,0.0
6,769176,190,2.0,29.0,0,0,0,13,3,12.0,...,0.426491,54264254,51003473,13942809,5191,4,41,19,0.0,0.0
7,1344732,125,2.0,29.0,0,0,0,1,3,55.0,...,0.414979,958944,51003473,37724583,3733,18,30,0,0.0,0.0
10,56,89,99.0,656.0,0,0,0,7,3,12.0,...,0.785251,6650374,51003473,13942809,5,4,28,7,0.0,0.0


In [16]:
train.columns

Index(['a_follower_count', 'a_following_count', 'b_follower_count',
       'b_following_count', 'hashtags_count_t', 'domains_count_t',
       'links_count_t', 'media', 'tweet_type', 'language', 'a_is_verified',
       'b_is_verified', 'b_follows_a', 'TE_media_reply', 'TE_media_retweet',
       'TE_media_retweet_comment', 'TE_media_like', 'TE_tweet_type_reply',
       'TE_tweet_type_retweet', 'TE_tweet_type_retweet_comment',
       'TE_tweet_type_like', 'TE_language_reply', 'TE_language_retweet',
       'TE_language_retweet_comment', 'TE_language_like', 'TE_a_user_id_reply',
       'TE_a_user_id_retweet', 'TE_a_user_id_retweet_comment',
       'TE_a_user_id_like', 'TE_b_user_id_reply', 'TE_b_user_id_retweet',
       'TE_b_user_id_retweet_comment', 'TE_b_user_id_like',
       'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_reply',
       'TE_domains_language_b_follows_a_tweet_type_media_a_is_verified_retweet',
       'TE_domains_language_b_follows_a_tweet_type_media_a_is_

In [17]:
%%time
# TRAIN AND VALIDATE

NROUND = 300
VERBOSE_EVAL = 50
#ESR = 50
    
oof = np.zeros((len(valid),len(label_names)))
preds = []
for i in range(4):

    name = label_names[i]
    print('#'*25);print('###',name);print('#'*25)
       
    start = time.time(); print('Creating DMatrix...')
    dtrain = xgb.dask.DaskDMatrix(client,data=train,label=Y_train.iloc[:, i])
    dvalid = xgb.dask.DaskDMatrix(client,data=valid,label=Y_valid.iloc[:, i])
    print('Took %.1f seconds'%(time.time()-start))
             
    start = time.time(); print('Training...')
    model = xgb.dask.train(client, xgb_parms, 
                           dtrain=dtrain,
                           #evals=[(dtrain,'train'),(dvalid,'valid')],
                           num_boost_round=NROUND,
                           #early_stopping_rounds=ESR,
                           verbose_eval=VERBOSE_EVAL) 
    print('Took %.1f seconds'%(time.time()-start))
        
    start = time.time(); print('Predicting...')
    #Y_valid[f'pred_{name}'] = xgb.dask.predict(client,model,valid)
    #oof[:, i] += xgb.dask.predict(client,model,dvalid).compute()
    preds.append(xgb.dask.predict(client,model,valid))
    print('Took %.1f seconds'%(time.time()-start))
        
    del model, dtrain, dvalid

#########################
### reply
#########################
Creating DMatrix...
Took 0.2 seconds
Training...


/conda/envs/rapids/lib/python3.7/site-packages/distributed/client.py:3498: RuntimeWarning: coroutine 'Client._update_scheduler_info' was never awaited
  self.sync(self._update_scheduler_info)


Took 15.1 seconds
Predicting...


/conda/envs/rapids/lib/python3.7/site-packages/distributed/worker.py:3385: UserWarning: Large object of size 4.44 MB detected in task graph: 
  [<function _predict_async.<locals>.mapped_predict  ... titions>, True]
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


Took 0.5 seconds
#########################
### retweet
#########################
Creating DMatrix...
Took 0.2 seconds
Training...


...

Took 14.0 seconds
Predicting...
Took 0.6 seconds
#########################
### retweet_comment
#########################
Creating DMatrix...
Took 0.2 seconds
Training...


/conda/envs/rapids/lib/python3.7/site-packages/distributed/client.py:3498: RuntimeWarning: coroutine 'Client._update_scheduler_info' was never awaited
  self.sync(self._update_scheduler_info)


Took 13.8 seconds
Predicting...
Took 0.5 seconds
#########################
### like
#########################
Creating DMatrix...
Took 0.2 seconds
Training...


/conda/envs/rapids/lib/python3.7/site-packages/distributed/client.py:3498: RuntimeWarning: coroutine 'Client._update_scheduler_info' was never awaited
  self.sync(self._update_scheduler_info)


Took 14.4 seconds
Predicting...
Took 0.6 seconds
CPU times: user 5.31 s, sys: 2.09 s, total: 7.4 s
Wall time: 1min


In [18]:
yvalid = Y_valid[label_names].values.compute()
oof = cupy.array([i.values.compute() for i in preds]).T

In [19]:
from sklearn.metrics import auc

def precision_recall_curve(y_true,y_pred):
    y_true = y_true.astype('float32')
    ids = cupy.argsort(-y_pred) 
    y_true = y_true[ids]
    y_pred = y_pred[ids]
    y_pred = cupy.flip(y_pred,axis=0)

    acc_one = cupy.cumsum(y_true)
    sum_one = cupy.sum(y_true)
    
    precision = cupy.flip(acc_one/cupy.cumsum(cupy.ones(len(y_true))),axis=0)
    precision[:-1] = precision[1:]
    precision[-1] = 1.

    recall = cupy.flip(acc_one/sum_one,axis=0)
    recall[:-1] = recall[1:]
    recall[-1] = 0
    n = (recall==1).sum()
    
    return precision[n-1:],recall[n-1:],y_pred[n:]

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    recall, prec = cupy.asnumpy(recall), cupy.asnumpy(prec)
    prauc = auc(recall, prec)
    return prauc

def log_loss(y_true,y_pred,eps=1e-7, normalize=True, sample_weight=None):
    y_true = y_true.astype('int32')
    y_pred = cupy.clip(y_pred, eps, 1 - eps)
    if y_pred.ndim == 1:
        y_pred = cupy.expand_dims(y_pred, axis=1)
    if y_pred.shape[1] == 1:
        y_pred = cupy.hstack([1 - y_pred, y_pred])

    y_pred /= cupy.sum(y_pred, axis=1, keepdims=True)
    loss = -cupy.log(y_pred)[cupy.arange(y_pred.shape[0]), y_true]
    return _weighted_sum(loss, sample_weight, normalize).item()

def _weighted_sum(sample_score, sample_weight, normalize):
    if normalize:
        return cupy.average(sample_score, weights=sample_weight)
    elif sample_weight is not None:
        return cupy.dot(sample_score, sample_weight)
    else:
        return sample_score.sum()

# FAST METRIC FROM GIBA
def compute_rce_fast(pred, gt):
    cross_entropy = log_loss(gt, pred)
    yt = np.mean(gt)     
    strawman_cross_entropy = -(yt*np.log(yt) + (1 - yt)*np.log(1 - yt))
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

In [ ]:
# NV + NV3 (all TargetEncoding + count encoding)

txt = ''
for i in range(4):
    prauc = compute_prauc(oof[:,i], yvalid[:, i])#.item()
    rce   = compute_rce_fast(oof[:,i], yvalid[:, i]).item()
    txt_ = f"{label_names[i]:20} PRAUC:{prauc:.5f} RCE:{rce:.5f}"
    print(txt_)
    txt += txt_ + '\n'

reply                PRAUC:0.15568 RCE:17.91003
retweet              PRAUC:0.52569 RCE:28.39455
retweet_comment      PRAUC:0.05174 RCE:10.71767
